# Generators: x = yield 42 
## And other applications

### David Stuebe

<a href="https://www.swipely.com/" target="_blank">www.swipely.com</a>

<a href="mailto:davidstuebe@swipely.com">davidstuebe@swipely.com</a>

March 2, 2015

##What is a generator?
___

```
    A kind of function that can return an intermediate result ("the next
    value") to its caller, but maintaining the function's local state so
    that the function can be resumed again right where it left off.
```
<a href="https://www.python.org/dev/peps/pep-0255/" target="_blank">PEP 255</a> introduced the generator object and the *yield* statement in version 2.2 of Python.

In [243]:
def fib():
    a, b = 0, 1
    while True:
        yield b
        a, b = b, a+b

In [244]:
function_result = fib()
print function_result

<generator object fib at 0x10ea421e0>


---
The result of calling the gererator function is a generator object.
The generator object can be used as an iterator.

The generator is an iterator
---

In [245]:
g.__iter__() is g


True

In [246]:
zip(xrange(10),function_result)

[(0, 1),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 5),
 (5, 8),
 (6, 13),
 (7, 21),
 (8, 34),
 (9, 55)]

In [247]:
(10, function_result.next())

(10, 89)

## Inside a generator
---
Inside the generator we can see that execution is paused after the *yield* and state is maintained between calls to next


In [248]:
def noisy_generator():
    print 'Initializing'
    print 'first yield'
    yield 1
    print 'generator running...'
    print 'second yield'
    yield 2
    print 'generator running...'
    print 'now what?'
g = noisy_generator()        

In [249]:
g.next()

Initializing
first yield


1

In [250]:
g.next()

generator running...
second yield


2

In [251]:
g.next()

generator running...
now what?


StopIteration: 

## Generator interface: next
---

In [255]:
g.next.__doc__

'x.next() -> the next value, or raise StopIteration'

In [256]:
help(g.next)

Help on method-wrapper object:

next = class method-wrapper(object)
 |  Methods defined here:
 |  
 |  __call__(...)
 |      x.__call__(...) <==> x(...)
 |  
 |  __cmp__(...)
 |      x.__cmp__(y) <==> cmp(x,y)
 |  
 |  __getattribute__(...)
 |      x.__getattribute__('name') <==> x.name
 |  
 |  __hash__(...)
 |      x.__hash__() <==> hash(x)
 |  
 |  __repr__(...)
 |      x.__repr__() <==> repr(x)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __objclass__
 |  
 |  __self__



## Generator interface: throw
---

In [257]:
help(g.throw)

Help on built-in function throw:

throw(...)
    throw(typ[,val[,tb]]) -> raise exception in generator,
    return next yielded value or raise StopIteration.



Looks fun - lets try it...

In [230]:
g = noisy_generator()        
g.next()
g.throw(StandardError, 'Foo bar baz')

Initializing
first yield


StandardError: Foo bar baz

In [231]:
g.next()

StopIteration: 

## Generator interface: close
---

In [232]:
help(g.close)

Help on built-in function close:

close(...)
    close() -> raise GeneratorExit inside generator.



In [233]:
def closeable():
    try:
        yield 1
        yield 2
    except GeneratorExit:
        print 'closing'
g = closeable()
g.next()

1

In [234]:
g.close()

closing


In [235]:
g.next()

StopIteration: 

## Generator interface: send
---

In [236]:
help(g.send)

Help on built-in function send:

send(...)
    send(arg) -> send 'arg' into generator,
    return next yielded value or raise StopIteration.



In [237]:
g = fib()
g.send('foo')

TypeError: can't send non-None value to a just-started generator

so *next()* is really just *send(None)* 

## Lets try that again...

In [238]:
g = fib()
g.send(None)

1

In [239]:
g.send(None)

1

What is *send('foo')* ?

In [240]:
g.send('foo')

2

Where did it go?

## What is generator.send?
```
    Coroutines are a natural way of expressing many algorithms, such as
    simulations, games, asynchronous I/O, and other forms of event-
    driven programming or co-operative multitasking.  Python's generator
    functions are almost coroutines -- but not quite -- in that they
    allow pausing execution to produce a value, but do not provide for
    values or exceptions to be passed in when execution resumes.  They
    also do not allow execution to be paused within the "try" portion of
    try/finally blocks, and therefore make it difficult for an aborted
    coroutine to clean up after itself.
```
<a href="https://www.python.org/dev/peps/pep-0342/" target="_blank">PEP 342</a> added *close*, *send* and *throw* to the generator in version 2.5 of python and made *yield* an expresion rather than a statement.


In [241]:
def coroutine(func):
  """ A helper function decorator from Beazley"""
  def start(*args, **kwargs):
    g = func(*args, **kwargs)
    g.next()
    return g
  return start

@coroutine
def cotuple2list():
  """This does the work"""
  result = None
  while True:
    (tup, co_pool) = (yield result)
    result = list(tup)
    # I don't like using append. So I am changing the data in place.
    for (i,x) in enumerate(result):
      # consider using "if hasattr(x,'__iter__')"
      if isinstance(x,tuple):
        result[i] = co_pool[0].send((x, co_pool[1:]))


@coroutine
def colist2tuple():
  """This does the work"""
  result = None
  while True:
    (lst, co_pool) = (yield result)
    # I don't like using append so I am changing the data in place...
    for (i,x) in enumerate(lst):
      # consider using "if hasattr(x,'__iter__')"
      if isinstance(x,list):
        lst[i] = co_pool[0].send((x, co_pool[1:]))
    result = tuple(lst)

def list2tuple(a):
  return tuple((list2tuple(x) if isinstance(x, list) else x for x in a))

def tuple2list(a):
  return list((tuple2list(x) if isinstance(x, tuple) else x for x in a))


def make_test(m, n):
  # Test data function taken from HYRY's post!
  return [[range(m), make_test(m, n-1)] for i in range(n)]


In [ ]:
list2tuple(make_test(2,3))

### Based on David M. Beazley
### Generators: The Final Frontier
<a href="http://www.dabeaz.com/finalgenerator/" target="_blank">http://www.dabeaz.com/finalgenerator/</a>

### A Curious Course on Coroutines and Concurrency
<a href="http://www.dabeaz.com/coroutines/" target="_blank">http://www.dabeaz.com/coroutines/</a>
